In [6]:
# CSE 6242 Group 9 Project
# Python script to access injury timeseries data in text format build a db where every unique injury occurance is an individual row

import pandas as pd
import os
import re

cwd = os.getcwd()
csv_path = cwd + "/output_data"

# 0. Import data, do basic cleaning, and cast columns
df = pd.read_csv("input_data/injuries_2010-2020.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.drop_duplicates()
print('Original rows: '+str(len(df)))

Original rows: 27103


In [7]:
# 1. Drop where Acquired not NAN
df = df[~df['Acquired'].notna()]
print('Rows after 1: '+str(len(df)))

# 2. Drop Acquired column, rename Relinquished to Player, then drop any row with mention of 'activat'
df = df.drop(columns=['Acquired'])
df = df.rename(columns={"Relinquished": "Player"})
df = df[~df['Notes'].str.contains("activat")]
print('Rows after 2: '+str(len(df))) # at this point all return to IL type entries should be removed

# 3. Order by player, then date to see each player's timeline
df = df.sort_values(by=['Player', 'Date'])
df.to_csv(csv_path+"/injury_temp.csv", index=False)

Rows after 1: 17562
Rows after 2: 17560


In [16]:
# 4. Apply Maggie's OpenRefine logic to categorize into Injury, Illness, or Rest (sickness, etc) then re-read file
df = pd.read_csv("output_data/injury_temp2.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.drop_duplicates()
df = df = df[df['type'] == 'injury'] # only select injuries
print('Rows after 4: '+str(len(df)))

# 5. Apply algorithm to check for repeat or near entries

# Rules:
# - If dates are less than 5 days apart, only take the column without 

df["lag"] = df['Notes'].shift(1)
print(df)


Rows after 4: 15839
            Date   Team              Player  \
0     2013-03-01  Hawks          Mike Scott   
1     2014-11-12  Hawks          Mike Scott   
3     2015-03-13  Hawks          Mike Scott   
4     2015-04-15  Hawks          Mike Scott   
5     2015-12-09  Hawks          Mike Scott   
...          ...    ...                 ...   
17553 2015-02-02   Suns        Zoran Dragic   
17554 2015-03-09   Heat        Zoran Dragic   
17556 2011-05-15   Heat  Zydrunas Ilgauskas   
17557 2011-05-31   Heat  Zydrunas Ilgauskas   
17558 2018-12-19  Bucks                 NaN   

                                          Notes    type  \
0                                  placed on IL  injury   
1                 placed on IL with back injury  injury   
3      placed on IL with fractured left big toe  injury   
4                 placed on IL with back injury  injury   
5                             knee injury (DTD)  injury   
...                                         ...     ...   
17